In [2]:
# Dependencies
import requests
import pandas as pd
import matplotlib as plt
import numpy as np
import json

# Save url as variable
url = "https://us-states.p.rapidapi.com/all"

headers = {
	"X-RapidAPI-Key": "575ca32490msh5a85e1221a9daf0p10835bjsnfdf17e56a208",
	"X-RapidAPI-Host": "us-states.p.rapidapi.com"
}

response = requests.request("GET", url, headers=headers)
data = response.json()

print(response.text)

[{"name": "Alabama", "postal": "AL", "capital": {"name": "Montgomery", "latitude": "32.377716", "longitude": "-86.300568"}, "date": "Dec 14, 1819", "population": {"density_km": "37", "total": "5024279", "density_mi": "95"}, "nickname": "Heart of Dixie", "fips": "01", "demonym": "Alabamian", "cities": [{"name": "Huntsville", "population": "215006"}, {"name": "Birmingham", "population": "200733"}, {"name": "Montgomery", "population": "198956"}, {"name": "Mobile", "population": "189572"}, {"name": "Tuscaloosa", "population": "101113"}], "lowest_point": "Gulf of Mexico", "highest_point": "Cheaha Mountain", "elevation": {"min_ft": "sea level", "min_m": "sea level", "mean_ft": "500", "max_rank": "38", "max_ft": "2413", "span_ft": "2413", "mean_rank": "44", "span_m": "736", "mean_m": "150", "max_m": "736"}, "area": {"total_mi": "52420.07", "land_rank": "28", "land_km": "131171", "water_rank": "25", "total_rank": "30", "land_mi": "50645.33", "water_km": "4597", "total_km": "135767", "land_perc

In [3]:
# Write the data to a json file
with open('data.json', 'w') as outfile:
    json.dump(data, outfile, indent=4)

In [4]:
# Create a pandas data frame
df = pd.json_normalize(response.json())
df

,name,postal,date,nickname,fips,demonym,cities,lowest_point,highest_point,website,...,symbols.handgun,symbols.hymn,symbols.star,symbols.astronomical symbol,symbols.emblem,symbols.heritage livestock,symbols.bat,symbols.oyster,symbols.cocktail,symbols.domestic animal
0,Alabama,AL,"Dec 14, 1819",Heart of Dixie,01,Alabamian,"[{'name': 'Huntsville', 'population': '215006'...",Gulf of Mexico,Cheaha Mountain,http://www.alabama.gov,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Alaska,AK,"Jan 3, 1959",The Last Frontier,02,Alaskan,"[{'name': 'Anchorage', 'population': '291538'}...","Gulf of Alaska, Bering Sea, and Arctic Ocean",Denali,http://alaska.gov/,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,American Samoa,AS,"Feb 20, 1929",Motu o Fiafiaga,60,American Samoan,"[{'name': 'Tafuna', 'population': '7945'}, {'n...",Pacific Ocean,Lata Mountain on Ta‘ū island,https://www.americansamoa.gov/,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Arizona,AZ,"Feb 14, 1912",Grand Canyon State,04,Arizonan,"[{'name': 'Phoenix', 'population': '1660272'},...","Colorado River at Sonora border, San Luis, Ari...",Humphreys Peak,https://az.gov/,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Arkansas,AR,"Jun 15, 1836",Natural State,05,Arkansan,"[{'name': 'Little Rock', 'population': '197881...",Ouachita River at Louisiana border,Mount Magazine,http://www.arkansas.gov/,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,California,CA,"Sep 9, 1850",Golden State,06,Californian,"[{'name': 'Los Angeles', 'population': '399798...",Badwater Basin in Death Valley,Mount Whitney,https://www.ca.gov/,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,Colorado,CO,"Aug 1, 1876",Centennial State,08,Coloradan,"[{'name': 'Denver', 'population': '719238'}, {...",Arikaree River at Kansas border,Mount Elbert,http://www.colorado.gov/,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,Connecticut,CT,"Jan 9, 1788",Constitution State,09,Connecticuter,"[{'name': 'Bridgeport', 'population': '144900'...",Long Island Sound,Massachusetts border on the southern slope of ...,https://portal.ct.gov/,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,Delaware,DE,"Dec 7, 1787",First State,10,Delawarean,"[{'name': 'Wilmington', 'population': '70635'}...",Atlantic Ocean,Near the Ebright Azimuth,https://delaware.gov/,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,Florida,FL,"Mar 3, 1845",Sunshine State,12,Floridian,"[{'name': 'Jacksonville', 'population': '90795...",Atlantic Ocean and Gulf of Mexico,Britton Hill,http://myflorida.com/,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
# Check data types
df.dtypes

name                          object
postal                        object
date                          object
nickname                      object
fips                          object
                               ...  
symbols.heritage livestock    object
symbols.bat                   object
symbols.oyster                object
symbols.cocktail              object
symbols.domestic animal       object
Length: 203, dtype: object

In [6]:
# Check the counts
df.count()

name                          56
postal                        56
date                          56
nickname                      56
fips                          56
                              ..
symbols.heritage livestock     1
symbols.bat                    1
symbols.oyster                 1
symbols.cocktail               1
symbols.domestic animal        1
Length: 203, dtype: int64

In [17]:
df.iloc[:,196:203]

,symbols.astronomical symbol,symbols.emblem,symbols.heritage livestock,symbols.bat,symbols.oyster,symbols.cocktail,symbols.domestic animal
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
# Drop all territories
states_df = df.drop([2,11,36,41,47])
states_df

,name,postal,date,nickname,fips,demonym,cities,lowest_point,highest_point,website,...,symbols.handgun,symbols.hymn,symbols.star,symbols.astronomical symbol,symbols.emblem,symbols.heritage livestock,symbols.bat,symbols.oyster,symbols.cocktail,symbols.domestic animal
0,Alabama,AL,"Dec 14, 1819",Heart of Dixie,01,Alabamian,"[{'name': 'Huntsville', 'population': '215006'...",Gulf of Mexico,Cheaha Mountain,http://www.alabama.gov,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Alaska,AK,"Jan 3, 1959",The Last Frontier,02,Alaskan,"[{'name': 'Anchorage', 'population': '291538'}...","Gulf of Alaska, Bering Sea, and Arctic Ocean",Denali,http://alaska.gov/,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Arizona,AZ,"Feb 14, 1912",Grand Canyon State,04,Arizonan,"[{'name': 'Phoenix', 'population': '1660272'},...","Colorado River at Sonora border, San Luis, Ari...",Humphreys Peak,https://az.gov/,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Arkansas,AR,"Jun 15, 1836",Natural State,05,Arkansan,"[{'name': 'Little Rock', 'population': '197881...",Ouachita River at Louisiana border,Mount Magazine,http://www.arkansas.gov/,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,California,CA,"Sep 9, 1850",Golden State,06,Californian,"[{'name': 'Los Angeles', 'population': '399798...",Badwater Basin in Death Valley,Mount Whitney,https://www.ca.gov/,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,Colorado,CO,"Aug 1, 1876",Centennial State,08,Coloradan,"[{'name': 'Denver', 'population': '719238'}, {...",Arikaree River at Kansas border,Mount Elbert,http://www.colorado.gov/,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,Connecticut,CT,"Jan 9, 1788",Constitution State,09,Connecticuter,"[{'name': 'Bridgeport', 'population': '144900'...",Long Island Sound,Massachusetts border on the southern slope of ...,https://portal.ct.gov/,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,Delaware,DE,"Dec 7, 1787",First State,10,Delawarean,"[{'name': 'Wilmington', 'population': '70635'}...",Atlantic Ocean,Near the Ebright Azimuth,https://delaware.gov/,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,Florida,FL,"Mar 3, 1845",Sunshine State,12,Floridian,"[{'name': 'Jacksonville', 'population': '90795...",Atlantic Ocean and Gulf of Mexico,Britton Hill,http://myflorida.com/,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10,Georgia,GA,"Jan 2, 1788",Peach State,13,Georgian,"[{'name': 'Atlanta', 'population': '504527'}, ...",Atlantic Ocean,Brasstown Bald,https://www.georgia.gov/,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [23]:
# Drop half of the columns that have nulls
clean_states_df = states_df.drop(columns=['status','other_nicknames','standard_federal_region','status','other_nicknames','standard_federal_region','koppen_climate','population.density_km','elevation.min_ft','elevation.min_m','elevation.mean_ft','elevation.span_ft','elevation.mean_rank','elevation.span_m','elevation.mean_m','elevation.max_m','area.land_km','area.water_rank','area.total_rank','area.land_mi','area.water_km','area.total_km','area.land_percent','area.water_mi','area.water_percent','symbols.mascot','symbols.fruit','symbols.tree fruit','symbols.soil','symbols.game bird','symbols.reptile','symbols.amphibian','symbols.nut','symbols.vegetable','symbols.rock','symbols.saltwater fish','symbols.mammal','symbols.butterfly','symbols.cake','symbols.insect','symbols.mineral','symbols.spirit','symbols.fossil','symbols.freshwater fish','symbols.wildflower','symbols.shell','symbols.land mammal','symbols.fish','symbols.bolt action rifle','symbols.dog','symbols.marine mammal','symbols.sport','symbols.plant','symbols.metal','symbols.colors','symbols.drink','symbols.firearm','symbols.dinosaur','symbols.floral emblem','symbols.folk dance','symbols.grape','symbols.fruit and vegetable','symbols.musical instrument','symbols.primitive fish','symbols.grain','symbols.cooking pot','symbols.lichen','symbols.grass','symbols.marine reptile','symbols.dance','symbols.tartan','symbols.gold rush ghost town', 'symbols.marine animal','symbols.macroinvertebrate','symbols.wildlife animal','symbols.dessert','symbols.bug','symbols.beverage','symbols.herb','symbols.saltwater mammal','symbols.heritage cattle breed','symbols.pie','symbols.stone','symbols.tortoise','symbols.saltwater reptile','symbols.prepared food','symbols.waltz','symbols.cold water gamefish','symbols.crop','symbols.raptor','symbols.exercise','symbols.artifact','symbols.snackfood','symbols.prairie grass','symbols.slogan','symbols.river','symbols.rifle','symbols.red wine grape','symbols.white wine grape','symbols.flying fossil','symbols.gun', 'symbols.wild animal game species',	'symbols.sports car', 'symbols.crustacean',	'symbols.meat pie','symbols.berry',	'symbols.treat','symbols.ballad','symbols.cat','symbols.team sport','symbols.donut','symbols.bean',	'symbols.cookie','symbols.muffin','symbols.folk song', 'symbols.bee','symbols.mushroom', 'symbols.waterfowl','symbols.toy','symbols.tree nut','symbols.fruit tree','symbols.hockey team','symbols.football team','symbols.endangered species','symbols.soft drink','symbols.poultry','symbols.saltwater game fish','symbols.freshwater game fish','symbols.spider','symbols.cryptid','symbols.bush','symbols.freshwater trout','symbols.salamander', 'symbols.marsupial','symbols.flying mammal','symbols.game animal','symbols.microbe','symbols.coral','symbols.appetizer','symbols.heritage horse','symbols.jewelry','symbols.bread','symbols.shrub','symbols.pastry','symbols.dish','symbols.handgun','symbols.hymn', 'symbols.astronomical symbol','symbols.emblem','symbols.heritage livestock','symbols.bat','symbols.oyster','symbols.cocktail','symbols.domestic animal','symbols.hero','symbols.shellfish',"symbols.children's flower",'symbols.heroine','symbols.polka','symbols.music','symbols.game mammal','symbols.aquatic animal','symbols.frog','symbols.star','symbols.horse','symbols.neckwear','symbols.silver rush ghost town','symbols.marine fish','symbols.fabric',	'symbols.pets',	'symbols.cactus','symbols.animal','population.density_mi','ap_abbreviation','gpo_abbreviation'])
clean_states_df

,name,postal,date,nickname,fips,demonym,cities,lowest_point,highest_point,website,per_capita_income,median_household_income,representatives,time_zones,motto,subdivisions,capital.name,capital.latitude,capital.longitude,population.total,elevation.max_rank,elevation.max_ft,area.total_mi,area.land_rank,census_bureau.region,census_bureau.division,symbols.gem,symbols.bird,symbols.song,symbols.tree,symbols.flower
0,Alabama,AL,"Dec 14, 1819",Heart of Dixie,01,Alabamian,"[{'name': 'Huntsville', 'population': '215006'...",Gulf of Mexico,Cheaha Mountain,http://www.alabama.gov,"$23,606","$49,861",7,[CST],Audemus jura nostra defendere,"[Autauga, Baldwin, Barbour, Bibb, Blount, Bull...",Montgomery,32.377716,-86.300568,5024279,38,2413,52420.07,28,South,East South Central,Star Blue Quartz,Yellowhammer,Alabama,Southern Longleaf Pine,Camellia
1,Alaska,AK,"Jan 3, 1959",The Last Frontier,02,Alaskan,"[{'name': 'Anchorage', 'population': '291538'}...","Gulf of Alaska, Bering Sea, and Arctic Ocean",Denali,http://alaska.gov/,"$33,062","$74,346",1,"[AKST, HST]",North to the Future,"[Aleutians East Borough, Aleutians West Census...",Juneau,58.301598,-134.420212,733391,1,20310,665384.04,1,West,Pacific,Jade,Willow Ptarmigan,Alaska's Flag,Sitka Spruce,Forget-Me-Not
3,Arizona,AZ,"Feb 14, 1912",Grand Canyon State,04,Arizonan,"[{'name': 'Phoenix', 'population': '1660272'},...","Colorado River at Sonora border, San Luis, Ari...",Humphreys Peak,https://az.gov/,"$25,715","$59,246",9,[MST],Ditat Deus,"[Apache, Cochise, Coconino, Gila, Graham, Gree...",Phoenix,33.448143,-112.096962,7151502,12,12637,113990.3,6,West,Mountain,Turquoise,Cactus Wren,"Arizona March Song,Arizona",Palo Verde,Saguaro Cactus Blossom
4,Arkansas,AR,"Jun 15, 1836",Natural State,05,Arkansan,"[{'name': 'Little Rock', 'population': '197881...",Ouachita River at Louisiana border,Mount Magazine,http://www.arkansas.gov/,"$22,883","$47,062",4,[CST],Regnat populus,"[Arkansas, Ashley, Baxter, Benton, Boone, Brad...",Little Rock,34.746613,-92.288986,3011524,37,2753,53178.55,27,South,West South Central,Diamond,Mockingbird,NaN,Pine Tree,NaN
5,California,CA,"Sep 9, 1850",Golden State,06,Californian,"[{'name': 'Los Angeles', 'population': '399798...",Badwater Basin in Death Valley,Mount Whitney,https://www.ca.gov/,"$30,441","$75,277",53,[PST],Eureka,"[Alameda, Alpine, Amador, Butte, Calaveras, Co...",Sacramento,38.576668,-121.493629,39538223,2,14505,163694.74,3,West,Pacific,Benitoite,California Quail,NaN,"Coast Redwood,Giant Sequoia",California Poppy
6,Colorado,CO,"Aug 1, 1876",Centennial State,08,Coloradan,"[{'name': 'Denver', 'population': '719238'}, {...",Arikaree River at Kansas border,Mount Elbert,http://www.colorado.gov/,"$32,357","$71,953",7,[MST],Nil sine numine,"[Adams, Alamosa, Arapahoe, Archuleta, Baca, Be...",Denver,39.739227,-104.984856,5773714,3,14440,104093.67,8,West,Mountain,Aquamarine,Lark Bunting,"Where the Columbines Grow,Rocky Mountain High",Colorado Blue Spruce,Rocky Mountain Columbine
7,Connecticut,CT,"Jan 9, 1788",Constitution State,09,Connecticuter,"[{'name': 'Bridgeport', 'population': '144900'...",Long Island Sound,Massachusetts border on the southern slope of ...,https://portal.ct.gov/,"$39,373","$76,348",5,[EST],Qui transtulit sustinet,"[Fairfield, Hartford, Litchfield, Middlesex, N...",Hartford,41.764046,-72.682198,3605944,39,2379,5543.41,48,Northeast,New England,NaN,American Robin,Yankee Doodle,Charter Oak,Mountain Laurel
8,Delaware,DE,"Dec 7, 1787",First State,10,Delawarean,"[{'name': 'Wilmington', 'population': '70635'}...",Atlantic Ocean,Near the Ebright Azimuth,https://delaware.gov/,"$30,488","$64,805",1,[EST],Liberty and independence,"[Kent, New Castle, Sussex]",Dover,39.157307,-75.519722,989948,54,447,2488.72,50,South,South Atlantic,NaN,Delaware Blue Hen,Our Delaware,American Holly,Peach Blossoms
9,Florida,FL,"Mar 3, 1845",Sunshine State,12,Floridian,"[{'name': 'Jacksonville', 'population': '90795...",Atlantic Ocean and Gulf of Mexico,Britton Hill,http://myflorida.com/,"

In [24]:
pd.options.display.max_columns = None
clean_states_df

,name,postal,date,nickname,fips,demonym,cities,lowest_point,highest_point,website,per_capita_income,median_household_income,representatives,time_zones,motto,subdivisions,capital.name,capital.latitude,capital.longitude,population.total,elevation.max_rank,elevation.max_ft,area.total_mi,area.land_rank,census_bureau.region,census_bureau.division,symbols.gem,symbols.bird,symbols.song,symbols.tree,symbols.flower
0,Alabama,AL,"Dec 14, 1819",Heart of Dixie,01,Alabamian,"[{'name': 'Huntsville', 'population': '215006'...",Gulf of Mexico,Cheaha Mountain,http://www.alabama.gov,"$23,606","$49,861",7,[CST],Audemus jura nostra defendere,"[Autauga, Baldwin, Barbour, Bibb, Blount, Bull...",Montgomery,32.377716,-86.300568,5024279,38,2413,52420.07,28,South,East South Central,Star Blue Quartz,Yellowhammer,Alabama,Southern Longleaf Pine,Camellia
1,Alaska,AK,"Jan 3, 1959",The Last Frontier,02,Alaskan,"[{'name': 'Anchorage', 'population': '291538'}...","Gulf of Alaska, Bering Sea, and Arctic Ocean",Denali,http://alaska.gov/,"$33,062","$74,346",1,"[AKST, HST]",North to the Future,"[Aleutians East Borough, Aleutians West Census...",Juneau,58.301598,-134.420212,733391,1,20310,665384.04,1,West,Pacific,Jade,Willow Ptarmigan,Alaska's Flag,Sitka Spruce,Forget-Me-Not
3,Arizona,AZ,"Feb 14, 1912",Grand Canyon State,04,Arizonan,"[{'name': 'Phoenix', 'population': '1660272'},...","Colorado River at Sonora border, San Luis, Ari...",Humphreys Peak,https://az.gov/,"$25,715","$59,246",9,[MST],Ditat Deus,"[Apache, Cochise, Coconino, Gila, Graham, Gree...",Phoenix,33.448143,-112.096962,7151502,12,12637,113990.3,6,West,Mountain,Turquoise,Cactus Wren,"Arizona March Song,Arizona",Palo Verde,Saguaro Cactus Blossom
4,Arkansas,AR,"Jun 15, 1836",Natural State,05,Arkansan,"[{'name': 'Little Rock', 'population': '197881...",Ouachita River at Louisiana border,Mount Magazine,http://www.arkansas.gov/,"$22,883","$47,062",4,[CST],Regnat populus,"[Arkansas, Ashley, Baxter, Benton, Boone, Brad...",Little Rock,34.746613,-92.288986,3011524,37,2753,53178.55,27,South,West South Central,Diamond,Mockingbird,NaN,Pine Tree,NaN
5,California,CA,"Sep 9, 1850",Golden State,06,Californian,"[{'name': 'Los Angeles', 'population': '399798...",Badwater Basin in Death Valley,Mount Whitney,https://www.ca.gov/,"$30,441","$75,277",53,[PST],Eureka,"[Alameda, Alpine, Amador, Butte, Calaveras, Co...",Sacramento,38.576668,-121.493629,39538223,2,14505,163694.74,3,West,Pacific,Benitoite,California Quail,NaN,"Coast Redwood,Giant Sequoia",California Poppy
6,Colorado,CO,"Aug 1, 1876",Centennial State,08,Coloradan,"[{'name': 'Denver', 'population': '719238'}, {...",Arikaree River at Kansas border,Mount Elbert,http://www.colorado.gov/,"$32,357","$71,953",7,[MST],Nil sine numine,"[Adams, Alamosa, Arapahoe, Archuleta, Baca, Be...",Denver,39.739227,-104.984856,5773714,3,14440,104093.67,8,West,Mountain,Aquamarine,Lark Bunting,"Where the Columbines Grow,Rocky Mountain High",Colorado Blue Spruce,Rocky Mountain Columbine
7,Connecticut,CT,"Jan 9, 1788",Constitution State,09,Connecticuter,"[{'name': 'Bridgeport', 'population': '144900'...",Long Island Sound,Massachusetts border on the southern slope of ...,https://portal.ct.gov/,"$39,373","$76,348",5,[EST],Qui transtulit sustinet,"[Fairfield, Hartford, Litchfield, Middlesex, N...",Hartford,41.764046,-72.682198,3605944,39,2379,5543.41,48,Northeast,New England,NaN,American Robin,Yankee Doodle,Charter Oak,Mountain Laurel
8,Delaware,DE,"Dec 7, 1787",First State,10,Delawarean,"[{'name': 'Wilmington', 'population': '70635'}...",Atlantic Ocean,Near the Ebright Azimuth,https://delaware.gov/,"$30,488","$64,805",1,[EST],Liberty and independence,"[Kent, New Castle, Sussex]",Dover,39.157307,-75.519722,989948,54,447,2488.72,50,South,South Atlantic,NaN,Delaware Blue Hen,Our Delaware,American Holly,Peach Blossoms
9,Florida,FL,"Mar 3, 1845",Sunshine State,12,Floridian,"[{'name': 'Jacksonville', 'population': '90795...",Atlantic Ocean and Gulf of Mexico,Britton Hill,http://myflorida.com/,"

In [26]:
import requests

# Define the API endpoint and parameters
api_key = 'F02526DC-2E96-4574-B037-292EC2981EF5'
url = 'https://apps.bea.gov/api/data/'
params = {
    'UserID': api_key,
    'method': 'GetData',
    'datasetname': 'Regional',
    'TableName': 'SAGDP2N',
    'LineCode': '1',
    'GeoFips': 'STATE',
    'Year': 'LAST',
    'ResultFormat': 'JSON'
}

# Make a request to the BEA API
response = requests.get(url, params=params)
data = response.json()

# Iterate over the GDP data for each state
for item in data['BEAAPI']['Results']['Data']:
    state = item['GeoName']
    gdp = item['DataValue']
    print(f'{state}: {gdp}')





KeyError: 'Data'

In [46]:
import requests

url = "https://jsearch.p.rapidapi.com/search"

querystring = {"query":"Entry Level Data Scientist in USA","num_pages":"20"}

headers = {
	"X-RapidAPI-Key": "575ca32490msh5a85e1221a9daf0p10835bjsnfdf17e56a208",
	"X-RapidAPI-Host": "jsearch.p.rapidapi.com"
}

response = requests.request("GET", url, headers=headers, params=querystring)
entry_level_data_scientists = response.json()

# Write the data to a json file
with open('entry_level_data_scientists.json', 'w') as outfile:
    json.dump(entry_level_data_scientists, outfile, indent=4)



print(response.text)

{"status":"OK","request_id":"714f0232-4ec6-4e66-8991-756caddae3d9","data":[{"employer_name":"National Security Agency","employer_logo":"https://upload.wikimedia.org/wikipedia/commons/8/8d/Seal_of_the_U.S._National_Security_Agency.svg","employer_website":"http://www.nsa.gov","job_publisher":"LinkedIn","job_id":"LqfiD7It-VoAAAAAAAAAAA==","job_employment_type":"FULLTIME","job_title":"Data Scientist - Entry to Expert Level (Maryland Location)","job_apply_link":"https://www.linkedin.com/jobs/view/data-scientist-entry-to-expert-level-maryland-location-at-national-security-agency-3416708943","job_description":"Responsibilities\n\nData science at the National Security Agency (NSA) is a multi-disciplinary field that uses elements of mathematics, statistics, computer science, and application-specific knowledge to gather, make, and communicate principled conclusions from data. Data science is present in every aspect of the mission. NSA Data Scientists tackle challenging real-world problems levera

In [47]:
import requests

url = "https://jsearch.p.rapidapi.com/search"

querystring = {"query":"Entry Level Data Analyst in USA","num_pages":"20"}

headers = {
	"X-RapidAPI-Key": "575ca32490msh5a85e1221a9daf0p10835bjsnfdf17e56a208",
	"X-RapidAPI-Host": "jsearch.p.rapidapi.com"
}

response = requests.request("GET", url, headers=headers, params=querystring)
entry_level_data_analyst = response.json()

# Write the data to a json file
with open('entry_level_data_analyst.json', 'w') as outfile:
    json.dump(entry_level_data_analyst, outfile, indent=4)



print(response.text)

{"status":"OK","request_id":"2c8f41cb-c5b5-4007-94af-a19a8844520e","data":[{"employer_name":"SynergisticIT","employer_logo":"https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcS-UrAK1LCetSvno8PT3nMB4vJV4VdEEPN7PrgC&s=0","employer_website":null,"job_publisher":"LinkedIn","job_id":"yhoOS-HZt2kAAAAAAAAAAA==","job_employment_type":"FULLTIME","job_title":"Entry Level Data Analyst","job_apply_link":"https://www.linkedin.com/jobs/view/entry-level-data-analyst-at-synergisticit-3415228708","job_description":"About Us\n\nSynergistic IT is a full-service staffing and placement firm servicing client in America for the past 12+ years. We are dedicated towards fulfilling the IT needs of our clients. From staffing to full implementation of projects we provide the highest quality IT Services. We Intend to deliver exceptional student outcome. We don't just help you secure a tech job but build a solid career in technology.\n\nJob Description\n\nRoles and Responsibilities\n• Managing master data, incl

In [57]:
analyst_df = pd.json_normalize(entry_level_data_analyst, 'data',)
analyst_df

,employer_name,employer_logo,employer_website,job_publisher,job_id,job_employment_type,job_title,job_apply_link,job_description,job_is_remote,job_posted_at_timestamp,job_posted_at_datetime_utc,job_city,job_state,job_country,job_latitude,job_longitude,job_benefits,job_google_link,job_offer_expiration_datetime_utc,job_offer_expiration_timestamp,job_required_skills,job_experience_in_place_of_education,job_min_salary,job_max_salary,job_salary_currency,job_salary_period,job_required_experience.no_experience_required,job_required_experience.required_experience_in_months,job_required_experience.experience_mentioned,job_required_experience.experience_preferred,job_required_education.postgraduate_degree,job_required_education.professional_certification,job_required_education.high_school,job_required_education.associates_degree,job_required_education.bachelors_degree,job_required_education.degree_mentioned,job_required_education.degree_preferred,job_required_education.professional_certification_mentioned,job_highlights.Qualifications,job_highlights.Responsibilities,job_highlights.Benefits
0,SynergisticIT,https://encrypted-tbn0.gstatic.com/images?q=tb...,None,LinkedIn,yhoOS-HZt2kAAAAAAAAAAA==,FULLTIME,Entry Level Data Analyst,https://www.linkedin.com/jobs/view/entry-level...,About Us\n\nSynergistic IT is a full-service s...,False,1672614440,2023-01-01T23:07:20.000Z,Normal,IL,US,40.514202,-88.990630,None,https://www.google.com/search?gl=us&hl=en&rciv...,2023-01-31T23:07:30.000Z,1.675206e+09,None,False,NaN,NaN,None,None,False,NaN,False,False,False,False,False,False,True,True,True,False,"[Bachelors, Masters in Computer Science/ Compu...","[Managing master data, including creation, upd...","[On Job Technical support, Full time position]"
1,"Staffigo Technical Services, LLC",None,None,ZipRecruiter,WoR9Wg400KwAAAAAAAAAAA==,FULLTIME,Entry Level Data Analyst,https://www.ziprecruiter.com/c/Staffigo-Techni...,About us\n\nStaffigo is in the business of pro...,False,1649817968,2022-04-13T02:46:08.000Z,Roswell,GA,US,34.023243,-84.361560,None,https://www.google.com/search?gl=us&hl=en&rciv...,2023-02-21T00:00:00.000Z,1.676938e+09,None,False,NaN,NaN,None,None,False,NaN,True,True,False,False,False,False,False,True,True,False,"[Masters degree in Computer Science, Computer ...","[Help to prepare good quality data, Respond to...",NaN
2,Staffingly,https://encrypted-tbn0.gstatic.com/images?q=tb...,None,LinkedIn,pu4ks3ytBz0AAAAAAAAAAA==,FULLTIME,Full time - Entry level Data Analyst (Remote),https://www.linkedin.com/jobs/view/full-time-e...,Taking care of people is at the heart of every...,True,1674013848,2023-01-18T03:50:48.000Z,Denver,CO,US,39.739235,-104.990250,"[dental_coverage, health_insurance, retirement...",https://www.google.com/search?gl=us&hl=en&rciv...,2023-02-17T03:50:48.000Z,1.676606e+09,None,False,NaN,NaN,None,None,False,NaN,True,True,False,False,False,False,True,True,False,False,"[Physical: Computer keyboarding, Auditory/visu...",[Compiles data; prepares and distributes repor...,[Career development and promotional growth opp...
3,TRESUME,https://encrypted-tbn0.gstatic.com/images?q=tb...,None,Glassdoor,XQDqtWFNlzsAAAAAAAAAAA==,FULLTIME,Entry Level Data Analyst,https://www.glassdoor.com/job-listing/entry-le...,We are seeking an Entry level Data Analyst to ...,False,1672531200,2023-01-01T00:00:00.000Z,Reston,VA,US,38.958630,-77.357000,None,https://www.google.com/search?gl=us&hl=en&rciv...,2023-02-22T00:00:00.000Z,1.677024e+09,None,False,NaN,NaN,None,None,False,NaN,True,True,False,False,False,False,False,True,False,False,"[Bachelor's degree or relevant, 0-2 years of e...",[You will analyze data to understand business ...,"[Pay: $27.00 - $30.00 per hour, 8 hour shift]"
4,"Staffigo Technical Services, LLC",None,None,ZipRecruiter,1xKRzZwkq_EAAAAAAAAAAA==,FULLTIME,Entry Level Data Analyst,https://www.ziprecruiter.com/c/Staffigo-Techni...,About us\n\nStaffigo is in the business of pro...,False,1649817938,2022-04-13T02:45:38.000Z,Weehawken,NJ,US,40.766373

In [62]:
columns = ["employer_name","job_title","job_apply_link","job_description","job_is_remote","job_city","job_state","job_country","job_latitude","job_longitude"]
df = pd.json_normalize(entry_level_data_analyst, 'data')
new_df = df.loc[:, columns]
new_df

,employer_name,job_title,job_apply_link,job_description,job_is_remote,job_city,job_state,job_country,job_latitude,job_longitude
0,SynergisticIT,Entry Level Data Analyst,https://www.linkedin.com/jobs/view/entry-level...,About Us\n\nSynergistic IT is a full-service s...,False,Normal,IL,US,40.514202,-88.990630
1,"Staffigo Technical Services, LLC",Entry Level Data Analyst,https://www.ziprecruiter.com/c/Staffigo-Techni...,About us\n\nStaffigo is in the business of pro...,False,Roswell,GA,US,34.023243,-84.361560
2,Staffingly,Full time - Entry level Data Analyst (Remote),https://www.linkedin.com/jobs/view/full-time-e...,Taking care of people is at the heart of every...,True,Denver,CO,US,39.739235,-104.990250
3,TRESUME,Entry Level Data Analyst,https://www.glassdoor.com/job-listing/entry-le...,We are seeking an Entry level Data Analyst to ...,False,Reston,VA,US,38.958630,-77.357000
4,"Staffigo Technical Services, LLC",Entry Level Data Analyst,https://www.ziprecruiter.com/c/Staffigo-Techni...,About us\n\nStaffigo is in the business of pro...,False,Weehawken,NJ,US,40.766373,-74.025370
...,...,...,...,...,...,...,...,...,...,...
67,"Staffigo Technical Services, LLC",Entry Level Data Analyst,https://us.trabajo.org/job-1275-20230120-45fd0...,About us\n\nStaffigo is in the business of pro...,False,Fort Wayne,IN,US,41.079273,-85.139350
68,Staffigo Technical Services LLC,Entry Level Data Analyst,https://www.jobilize.com/job/us-ga-atlanta-ent...,About us\n\nStaffigo is in the business of pro...,False,Atlanta,GA,US,33.748753,-84.387690
69,"Staffigo Technical Services, LLC",Entry Level Data Analyst,https://maryland.tarta.ai/j/PNU2LoABCr2uuunQDN...,About Us\n\nStaffigo is in the business of pro...,False,Annapolis,MD,US,38.978447,-76.492180
70,"Staffigo Technical Services, LLC",Entry Level Data Analyst,https://us.trabajo.org/job-640-20230118-aee169...,About us\n\nStaffigo is in the business of pro...,False,Broomfield,CO,US,39.920540,-105.086650


In [63]:
data_scientist_df = pd.json_normalize(entry_level_data_scientists, 'data',)
data_scientist_df

,employer_name,employer_logo,employer_website,job_publisher,job_id,job_employment_type,job_title,job_apply_link,job_description,job_is_remote,job_posted_at_timestamp,job_posted_at_datetime_utc,job_city,job_state,job_country,job_latitude,job_longitude,job_benefits,job_google_link,job_offer_expiration_datetime_utc,job_offer_expiration_timestamp,job_required_skills,job_experience_in_place_of_education,job_min_salary,job_max_salary,job_salary_currency,job_salary_period,job_required_experience.no_experience_required,job_required_experience.required_experience_in_months,job_required_experience.experience_mentioned,job_required_experience.experience_preferred,job_required_education.postgraduate_degree,job_required_education.professional_certification,job_required_education.high_school,job_required_education.associates_degree,job_required_education.bachelors_degree,job_required_education.degree_mentioned,job_required_education.degree_preferred,job_required_education.professional_certification_mentioned,job_highlights.Qualifications,job_highlights.Responsibilities,job_highlights.Benefits
0,National Security Agency,https://upload.wikimedia.org/wikipedia/commons...,http://www.nsa.gov,LinkedIn,LqfiD7It-VoAAAAAAAAAAA==,FULLTIME,Data Scientist - Entry to Expert Level (Maryla...,https://www.linkedin.com/jobs/view/data-scient...,Responsibilities\n\nData science at the Nation...,False,1674301046,2023-01-21T11:37:26.000Z,Fort Meade,MD,US,39.104263,-76.734240,None,https://www.google.com/search?gl=us&hl=en&rciv...,2023-04-10T00:00:00.000Z,1.681085e+09,None,False,NaN,NaN,None,None,True,24.0,True,True,False,False,False,False,True,True,True,False,[Applicants who meet minimum qualifications ma...,"[As a Data Scientist, your responsibilities ma...","[Pay, Benefits, & Work Schedule, Salary offers..."
1,Eurofins USA BioPharma Services,None,None,Adzuna,TIZd1xbrDC0AAAAAAAAAAA==,FULLTIME,Entry Level Data Scientist,https://www.adzuna.com/details/3862859345,Job Description\n\nNOT A CODING OR DATA SCIENC...,False,1674171570,2023-01-19T23:39:30.000Z,None,PA,US,41.203323,-77.194530,None,https://www.google.com/search?gl=us&hl=en&rciv...,2023-02-03T21:49:47.000Z,1.675461e+09,None,False,NaN,NaN,None,None,False,NaN,False,False,False,False,False,False,False,False,False,False,NaN,[Determine if data is compliant and defendable...,NaN
2,SynergisticIT,https://encrypted-tbn0.gstatic.com/images?q=tb...,None,BeBee,vgdO2MFDMv4AAAAAAAAAAA==,FULLTIME,Entry level data scientist,https://us.bebee.com/job/20230123-c82ed72184ca...,SYNERGISTICIT wants every candidate to know we...,False,1674496485,2023-01-23T17:54:45.000Z,None,None,US,37.090240,-95.712890,None,https://www.google.com/search?gl=us&hl=en&rciv...,2023-01-31T00:00:00.000Z,1.675123e+09,None,False,NaN,NaN,None,None,False,12.0,True,False,False,False,False,False,False,True,True,False,[Who Should Apply Recent IT Graduates looking ...,NaN,NaN
3,SynergisticIT,https://encrypted-tbn0.gstatic.com/images?q=tb...,None,LinkedIn,p_C7keyJwVMAAAAAAAAAAA==,CONTRACTOR,ENTRY LEVEL DATA SCIENTIST,https://www.linkedin.com/jobs/view/entry-level...,SYNERGISTICIT wants every candidate to know th...,False,1674277640,2023-01-21T05:07:20.000Z,Oklahoma City,OK,US,35.467560,-97.516426,None,https://www.google.com/search?gl=us&hl=en&rciv...,2023-02-20T05:07:21.000Z,1.676870e+09,None,False,NaN,NaN,None,None,False,NaN,True,False,False,False,False,False,True,True,True,False,[Get the right opportunity and gain experience...,[Or their resume can be sent out to clients to...,NaN
4,SynergisticIT,https://encrypted-tbn0.gstatic.com/images?q=tb...,None,LinkedIn,Hb-zr__hIa8AAAAAAAAAAA==,PARTTIME,Entry Level Data Scientist,https://www.linkedin.com/jobs/view/entry-level...,"At SynergisticIT, we aim to bring aboard IT pr...",False,1673672858,2023-01-14T05:07:38.000Z,Seattle,WA,US,47.606210,-122.332070,None,https://www.google.com/search?gl=us&hl=en&rciv...,2023-02-13T05:07:38.000Z,1.676265e+09,None,False,NaN,NaN,None,None,False,NaN,True,False,False,False,False,Fals

In [64]:
columns = ["employer_name","job_title","job_apply_link","job_description","job_is_remote","job_city","job_state","job_country","job_latitude","job_longitude"]
df2 = pd.json_normalize(entry_level_data_scientists, 'data')
new_data_df = df2.loc[:, columns]
new_data_df

,employer_name,job_title,job_apply_link,job_description,job_is_remote,job_city,job_state,job_country,job_latitude,job_longitude
0,National Security Agency,Data Scientist - Entry to Expert Level (Maryla...,https://www.linkedin.com/jobs/view/data-scient...,Responsibilities\n\nData science at the Nation...,False,Fort Meade,MD,US,39.104263,-76.734240
1,Eurofins USA BioPharma Services,Entry Level Data Scientist,https://www.adzuna.com/details/3862859345,Job Description\n\nNOT A CODING OR DATA SCIENC...,False,None,PA,US,41.203323,-77.194530
2,SynergisticIT,Entry level data scientist,https://us.bebee.com/job/20230123-c82ed72184ca...,SYNERGISTICIT wants every candidate to know we...,False,None,None,US,37.090240,-95.712890
3,SynergisticIT,ENTRY LEVEL DATA SCIENTIST,https://www.linkedin.com/jobs/view/entry-level...,SYNERGISTICIT wants every candidate to know th...,False,Oklahoma City,OK,US,35.467560,-97.516426
4,SynergisticIT,Entry Level Data Scientist,https://www.linkedin.com/jobs/view/entry-level...,"At SynergisticIT, we aim to bring aboard IT pr...",False,Seattle,WA,US,47.606210,-122.332070
5,SynergisticIT,Entry Level Data Scientist,https://www.linkedin.com/jobs/view/entry-level...,About Us\n\nSynergistic IT is a full-service s...,False,Lansing,MI,US,42.732536,-84.555534
6,Eurofins USA BioPharma Services,Entry Level Data Scientist,https://www.monster.com/job-openings/entry-lev...,Company Description\n\nNOT A CODING OR DATA SC...,False,Lancaster,PA,US,40.037876,-76.305510
7,SynergisticIT,Entry Level Data Scientist,https://us.trabajo.org/job-1275-20230124-9c5ff...,SYNERGISTICIT wants every candidate to know th...,False,Aberdeen,SD,US,45.464700,-98.486480
8,SynergisticIT,Entry Level Data Scientist (Remote),https://www.datayoshi.com/offer/672278/entry-l...,Synergistic IT is a full-service staffing and ...,False,College Station,TX,US,30.627977,-96.334404
9,SynergisticIT,Entry Level Data Scientist,https://www.linkedin.com/jobs/view/entry-level...,About Us\n\nSynergistic IT is a full-service s...,False,Portland,ME,US,43.659100,-70.256820


In [66]:
result_df = pd.merge(new_df, new_data_df, on='employer_name')
result_df

,employer_name,job_title_x,job_apply_link_x,job_description_x,job_is_remote_x,job_city_x,job_state_x,job_country_x,job_latitude_x,job_longitude_x,job_title_y,job_apply_link_y,job_description_y,job_is_remote_y,job_city_y,job_state_y,job_country_y,job_latitude_y,job_longitude_y
0,SynergisticIT,Entry Level Data Analyst,https://www.linkedin.com/jobs/view/entry-level...,About Us\n\nSynergistic IT is a full-service s...,False,Normal,IL,US,40.514202,-88.990630,Entry level data scientist,https://us.bebee.com/job/20230123-c82ed72184ca...,SYNERGISTICIT wants every candidate to know we...,False,None,None,US,37.090240,-95.712890
1,SynergisticIT,Entry Level Data Analyst,https://www.linkedin.com/jobs/view/entry-level...,About Us\n\nSynergistic IT is a full-service s...,False,Normal,IL,US,40.514202,-88.990630,ENTRY LEVEL DATA SCIENTIST,https://www.linkedin.com/jobs/view/entry-level...,SYNERGISTICIT wants every candidate to know th...,False,Oklahoma City,OK,US,35.467560,-97.516426
2,SynergisticIT,Entry Level Data Analyst,https://www.linkedin.com/jobs/view/entry-level...,About Us\n\nSynergistic IT is a full-service s...,False,Normal,IL,US,40.514202,-88.990630,Entry Level Data Scientist,https://www.linkedin.com/jobs/view/entry-level...,"At SynergisticIT, we aim to bring aboard IT pr...",False,Seattle,WA,US,47.606210,-122.332070
3,SynergisticIT,Entry Level Data Analyst,https://www.linkedin.com/jobs/view/entry-level...,About Us\n\nSynergistic IT is a full-service s...,False,Normal,IL,US,40.514202,-88.990630,Entry Level Data Scientist,https://www.linkedin.com/jobs/view/entry-level...,About Us\n\nSynergistic IT is a full-service s...,False,Lansing,MI,US,42.732536,-84.555534
4,SynergisticIT,Entry Level Data Analyst,https://www.linkedin.com/jobs/view/entry-level...,About Us\n\nSynergistic IT is a full-service s...,False,Normal,IL,US,40.514202,-88.990630,Entry Level Data Scientist,https://us.trabajo.org/job-1275-20230124-9c5ff...,SYNERGISTICIT wants every candidate to know th...,False,Aberdeen,SD,US,45.464700,-98.486480
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
133,ERM,Research Analyst (Entry Level),https://www.linkedin.com/jobs/view/research-an...,"Shelton Group, an ERM Group company, exists to...",False,Charlotte,NC,US,35.227085,-80.843124,Staff Environmental Data Scientist (Entry Level),https://www.linkedin.com/jobs/view/staff-envir...,ERM is seeking a Staff Environmental Data Scie...,False,Charlotte,NC,US,35.227085,-80.843124
134,Verizon,Advanced Analytics Entry-Level Opportunities,https://www.verizon.com/about/work/jobs/111163...,When you join Verizon\n\nVerizon is one of the...,False,Boston,MA,US,42.360080,-71.058880,Advanced Analytics Entry-Level Opportunities,https://www.verizon.com/about/work/jobs/111163...,When you join Verizon\n\nVerizon is one of the...,False,Boston,MA,US,42.360080,-71.058880
135,National Security Agency,Data Scientist - Entry to Expert Level (Maryla...,https://www.linkedin.com/jobs/view/data-scient...,Responsibilities\n\nData science at the Nation...,False,Fort Meade,MD,US,39.104263,-76.734240,Data Scientist - Entry to Expert Level (Maryla...,https://www.linkedin.com/jobs/view/data-scient...,Responsibilities\n\nData science at the Nation...,False,Fort Meade,MD,US,39.104263,-76.734240
136,Synergistic IT,Entry Level Data Analyst,https://jooble.org/jdp/2426417117652925034/Ent...,SYNERGISTICIT wants every candidate to know th...,False,Atlanta,GA,US,33.748753,-84.387690,Entry level data scientist,https://sunnyvale-ca.geebo.com/jobs-online/vie...,SYNERGISTICIT wants every candidate to know th...,False,Sunnyvale,CA,US,37.368830,-122.036350
